In [ ]:
# Install required packages
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.9.2
    Uninstalling nbformat-5.9.2:
      Successfully uninstalled nb

In [ ]:
!pip install pointnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 918.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio


In [ ]:
!git clone https://github.com/yanx27/Pointnet_Pointnet2_pytorch.git


Cloning into 'Pointnet_Pointnet2_pytorch'...
remote: Enumerating objects: 842, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 842 (delta 7), reused 11 (delta 5), pack-reused 826
Receiving objects: 100% (842/842), 68.77 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (485/485), done.
Updating files: 100% (59/59), done.


In [ ]:
%cd Pointnet_Pointnet2_pytorch


/content/Pointnet_Pointnet2_pytorch


In [ ]:
import zipfile

# Specify the path to the ZIP file and the target directory
zip_file_path = "/content/Earlyply.zip"
target_directory = "/content/"  # Replace with your desired directory

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_directory)


In [ ]:
import zipfile

# Specify the path to the ZIP file and the target directory
zip_file_path = "/content/Healthyply.zip"
target_directory = "/content/"  # Replace with your desired directory

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_directory)

In [ ]:
import zipfile

# Specify the path to the ZIP file and the target directory
zip_file_path = "/content/Lateply.zip"
target_directory = "/content/"  # Replace with your desired directory

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_directory)

In [ ]:
import numpy as np
import torch
import os
from torch.utils.data import Dataset
import open3d as o3d
from torch.utils.data import Dataset, DataLoader

def point_cloud_to_voxel_grid(point_cloud, grid_size):
    voxel_grid = np.zeros((grid_size, grid_size, grid_size))

    # Normalize and scale point cloud to fit into the voxel grid
    min_bound = np.min(point_cloud, axis=0)
    max_bound = np.max(point_cloud, axis=0)
    scale = max_bound - min_bound
    scale[scale == 0] = 1  # Avoid division by zero
    normalized_points = (point_cloud - min_bound) / scale

    # Scale to voxel grid size
    scaled_points = (normalized_points * (grid_size - 1)).astype(int)

    # Fill the voxel grid
    for point in scaled_points:
        x, y, z = point
        voxel_grid[x, y, z] = 1  # Mark the voxel as occupied

    return voxel_grid

class Custom3DDataset(Dataset):
    def __init__(self, root_dirs, grid_size=32):
        # root_dirs is a dictionary with labels as keys and folder paths as values
        # e.g., {'late': '/content/Lateply', 'healthy': '/content/Healthyply', 'early': '/content/Earlyply'}
        self.grid_size = grid_size
        self.data = []
        self.labels = []

        for label, root_dir in root_dirs.items():
            filenames = os.listdir(root_dir)
            for filename in filenames:
                file_path = os.path.join(root_dir, filename)
                self.data.append(file_path)
                if label == 'late':
                    self.labels.append(0)
                elif label == 'healthy':
                    self.labels.append(1)
                elif label == 'early':
                    self.labels.append(2)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path = self.data[idx]
        ply = o3d.io.read_point_cloud(file_path)
        points = np.asarray(ply.points)
        voxel_grid = point_cloud_to_voxel_grid(points, self.grid_size)
        voxel_grid_tensor = torch.tensor(voxel_grid, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        label = self.labels[idx]
        return voxel_grid_tensor, label

# Example usage
root_dirs = {
    'late': '/content/Lateply',
    'healthy': '/content/Healthyply',
    'early': '/content/Earlyply'
}
combined_dataset = Custom3DDataset(root_dirs, grid_size=32)



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Simple3DCNN(nn.Module):
    def __init__(self):
        super(Simple3DCNN, self).__init__()
        # First 3D convolutional layer, taking in 1 input channel (assuming grayscale),
        # outputting 32 convolutional features, with a kernel size of 3
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)

        # Second 3D convolutional layer
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)

        # Third 3D convolutional layer
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4 * 4, 512)  # Assuming the input volume is 32x32x32
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 3)  # Output 3 classes

    def forward(self, x):
        # Apply the first convolution and pool
        x = self.pool(F.relu(self.conv1(x)))

        # Apply the second convolution and pool
        x = self.pool(F.relu(self.conv2(x)))

        # Apply the third convolution and pool
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the output for the fully connected layers
        x = x.view(-1, 128 * 4 * 4 * 4)  # Flatten the tensor

        # Fully connected layers with dropout
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)

        # Final output layer
        x = self.fc3(x)
        return x

model = Simple3DCNN()


Orginal


In [ ]:
from sklearn.model_selection import train_test_split
train_size = int(0.7 * len(combined_dataset))
val_size = int(0.2 * len(combined_dataset))
test_size = len(combined_dataset) - train_size - val_size

train_dataset, temp_dataset = torch.utils.data.random_split(combined_dataset, [train_size, val_size + test_size])
val_dataset, test_dataset = torch.utils.data.random_split(temp_dataset, [val_size, test_size])

# Create DataLoaders for each set
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
num_epochs=1

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}")

Epoch 1/1, Training Loss: 1.0770259113551823, Validation Loss: 1.0771970472335815


In [ ]:
def evaluate_model_on_test_data(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Calculate accuracy on test data
test_accuracy = evaluate_model_on_test_data(model, test_loader)
print(f"Accuracy on test data: {test_accuracy}%")


Accuracy on test data: 75.6%


In [ ]:
def load_and_preprocess_ply(file_path, grid_size=32):
    ply = o3d.io.read_point_cloud(file_path)
    points = np.asarray(ply.points)
    voxel_grid = point_cloud_to_voxel_grid(points, grid_size)
    voxel_grid_tensor = torch.tensor(voxel_grid, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions
    return voxel_grid_tensor

def predict(model, file_path, grid_size=32):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        voxel_grid_tensor = load_and_preprocess_ply(file_path, grid_size)
        outputs = model(voxel_grid_tensor)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

# Path to your .ply file
ply_file_path = '/content/exported_1.ply'

# Make a prediction
predicted_class = predict(model, ply_file_path, grid_size=32)

# Print the predicted class
class_names = ['late', 'healthy', 'early']
print(f"Predicted Class: {class_names[predicted_class]}")


Predicted Class: late
